시중 스마트폰 상세 정보  
갓 입사한 신입사원 김모씨는 기획부서에 배치되었다. 첫 프로젝트로 신규 스마트폰 스펙 기획업무를 보조하게 되었다. 장고 끝에 김씨는 온라인 디지털 마켓 사이트 '다판다요'에서 판매중인 스마트폰 데이터를 수집하여 이를 분석하고 향후 프로젝트의 백데이터로 활용하기로 결심하였다.  

데이터: mobiles.csv 430 rows, 11 columns, UTF-8

변수명, 유형, 설명  
screen_size, string, 화면크기  
ROM, int, 저장공간용량  
RAM, int, RAM용량  
num_rear_camera, int, 후면 카메라 개수  
num_front_camera, int, 전면 카메라 개수  
battery_capacity, int,  배터리용량  
ratings, float, 평가 점수 평균  
num_of_ratings, int,  평가개수  
sales_price, int, 판매가격  
discount_percent, float, 할인율  
sales, float, 판매지수

필요한 라이브러리  
from sklearn.preprocessing import MinMaxScaler  
from sklearn.model_selection import train_test_split  
from sklearn.neighbors import KNeighborsRegressor  
from sklearn.metrics import mean_squared_error  

문제1. 스마트폰의 경우 많은 제품이 출시되지만, 정작 주목받는 제품은 극히 적다고 한다. 판매지수(sales)를 기준으로 이상치라고 판단되는 제품을 주목받는 제품이라고 판단하고 해당 제품들의 성능지표를 산출하시오. 산출된 성능지표의 평균은 얼마인가?  
<성능지표 계산식>  
성능지표 = (ROM/32) + (RAM/2) + 카메라 개수 + (배터리용량/1000)  
. 카메라 개수: num_rear_camera + num_front_camera  
. 이상치는 평균으로 부터 2표준편차보다 큰 값으로 정의한다.  
. 결과는 소수점 2째자리까지 계산

문제2. 판매지수(sales)와 가장 상관계수가 높은 변수를 찾고자 한다. 배터리 용량, 평가점수평균, 평가개수, 판매가격, 할인율 변수와 판매지수를 피어슨 상관분석을 실시하였을 때 상관계수의 절대값이 가장 큰 변수의 상관계수는 얼마인가?  
. 후면 카메라가 1개인 제품은 제외하시오.  
. 통계적 유의성은 고려하지 않음.  
. 결과는 반올림하여 소수점 2째자리까지...

문제3. 판매지수를 예측하기 위해 머신러닝 모델을 활용하고자 한다. k-NN알고리즘을 사용하고 연산에 사용하는 이웃의 개수를 변화하면서 가장 성능이 좋은 모델을 확보하려 한다. RMSE(root mean squared error)를 기준으로 가장 성능이 좋은 모델을 확인하고 해당 모델의 k(이웃 개수)를 구하라.  
. 독립변수: 판매지수를 제외한 모든 변수  
. 종속변수: 판매지수  
. 명목형 독립변수의 경우  one hot encoding을 실시한 결과를 모델에 사용하시오.  
. 학습에 사용하는 독ㄹ립변수의 개수는 총 14개이다.  
. 학습 및 평가 데이터 세트 분할비는 8:2  
. 정규화는 Min-Max정규화를 실시하며, 평가데이터세트는 학습 데이터 세트 기반으로 정규화 하시오.  
. seed는 123  
. 최근접 이웃은 3, 5, 7, 9, 11dmf tkdyd
. 정답은 자연수로 출력하시오

추가문제. 지난달 신규 스마트폰 출시됨. 해당 스마트폰의 판매지수는 얼마로 예상하는가?  
ROM, RAM, num_camera, battery_capacity, ratings, num_of_ratings, sales_prices, discount_percent, screen_size

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


### Q1.

In [3]:
stat_mean = df["sales"].mean()
stat_std  = df["sales"].std()
stat_out  = stat_mean + 2 * stat_std
stat_out

146.55150129273218

In [4]:
df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True)

In [5]:
len(df_q1)

16

In [6]:
df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
(df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + (df_q1["battery_capacity"] / 1000)

In [7]:
round(df_q1["idx"].mean(), 2)

11.01

### Q2.

In [8]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":]
df_q2.head(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
1,2815,4.5,244,57149,0.04,1.39
4,2815,4.6,745,69149,0.02,5.15


In [9]:
df_corr = df_q2.corr()
df_corr.abs().round(2)["sales"] # 0.95 <--

battery_capacity    0.03
ratings             0.23
num_of_ratings      0.95
sales_price         0.25
discount_percent    0.22
sales               1.00
Name: sales, dtype: float64

In [10]:
df_corr.replace(1, np.nan).abs().max().round(2)

battery_capacity    0.50
ratings             0.42
num_of_ratings      0.95
sales_price         0.50
discount_percent    0.26
sales               0.95
dtype: float64

In [11]:
df_corr_melt = df_corr.reset_index().melt(id_vars = "index")
df_corr_melt.loc[df_corr_melt["index"] != df_corr_melt["variable"], ]

,index,variable,value
1,ratings,battery_capacity,-0.424129
2,num_of_ratings,battery_capacity,0.034902
3,sales_price,battery_capacity,-0.503019
4,discount_percent,battery_capacity,0.257373
5,sales,battery_capacity,0.025680
6,battery_capacity,ratings,-0.424129
8,num_of_ratings,ratings,0.191655
9,sales_price,ratings,0.151153
10,discount_percent,ratings,-0.118578
11,sales,ratings,0.226075


In [12]:
df_corr_melt_sub = df_corr_melt.loc[df_corr_melt["index"] != df_corr_melt["variable"], ]
df_corr_melt_sub.pivot(index = "index", columns = "variable", values = "value")

variable,battery_capacity,discount_percent,num_of_ratings,ratings,sales,sales_price
index,,,,,,
battery_capacity,NaN,0.257373,0.034902,-0.424129,0.025680,-0.503019
discount_percent,0.257373,NaN,0.212442,-0.118578,0.223471,-0.098640
num_of_ratings,0.034902,0.212442,NaN,0.191655,0.949114,-0.260279
ratings,-0.424129,-0.118578,0.191655,NaN,0.226075,0.151153
sales,0.025680,0.223471,0.949114,0.226075,NaN,-0.247760
sales_price,-0.503019,-0.098640,-0.260279,0.151153,-0.247760,NaN


### Q3.

In [13]:
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


In [14]:
# df_q3_dum = pd.get_dummies(df, columns = ["screen_size"]) # 시험버전 기준
df_q3_dum = pd.get_dummies(df, dtype = "int") # Pandas 2.0.0 기준.
df_q3_dum.head(2)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,64,2,1,1,1800,4.5,38645,32999,0.17,127.52,0,0,0,0,1
1,64,4,2,1,2815,4.5,244,57149,0.04,1.39,0,0,1,0,0


"statsmodels" 라이브러리를 사용하여 모델링을 하는 경우 논리값(True/False)가 있으면 모델링시 에러가 발생하기 때문에 반드시 정수형으로 변환해주어야 한다. 그리고 상기 데이터프레임에는 "screen_size_Very Large" 라는 변수명이 있는데 이를 statsmodels기반의 모델링을 실시하는 경우(formula 사용시) 에러가 발생할 수 있다.

In [15]:
df_q3_dum.shape

(430, 15)

In [16]:
df_q3_dum = df_q3_dum.set_index("sales").reset_index()
df_q3_dum.head(2)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,127.52,64,2,1,1,1800,4.5,38645,32999,0.17,0,0,0,0,1
1,1.39,64,4,2,1,2815,4.5,244,57149,0.04,0,0,1,0,0


In [17]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8, random_state = 123)
len(df_train), len(df_test)

(344, 86)

In [18]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [19]:
arr_train_nor[:1, ]

array([[0.00394753, 0.04761905, 0.18181818, 0.33333333, 0.        ,
        0.42307692, 0.625     , 0.00396262, 0.02607261, 0.09302326,
        0.        , 0.        , 1.        , 0.        , 0.        ]])

In [ ]:
ls_k = [3, 5, 7, 9, 11]
k = ls_k[0]

model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = arr_train_nor[:, 1:],
              y = arr_train_nor[:, 0 ])
pred = model_knn.predict(arr_test_nor[:, 1:])

val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                              y_pred = pred) ** 0.5
val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                              y_pred = pred,
                              squared = False)
val_rmse

In [27]:
ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in ls_k:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = arr_train_nor[:, 1:],
                  y = arr_train_nor[:, 0 ])
    pred = model_knn.predict(arr_test_nor[:, 1:])

    val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                                  y_pred = pred) ** 0.5

    ls_rmse = ls_rmse + [val_rmse]

In [40]:
k

11

In [29]:
best_k = pd.Series(ls_rmse, index = ls_k).idxmin()
best_k

3

### Q3. 추가 지시사항
다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?  
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오  
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.  
* ROM: 256
* RAM: 6
* num_rear_camera: 4
* num_front_camera: 1
* battery_capacity: 4000
* ratings: 4.3
* num_of_ratings: 25000
* sales_price: 85000
* discount_percent: 0.05
* screen_size: "Large"


In [32]:
# df_s1 = pd.DataFrame(dict(ROM = 256, ~~~ ))
df_s1 = df_test.head(1).reset_index(drop = True)

In [33]:
df_s1["RAM"] = 6
df_s1["num_rear_camera"] = 4
df_s1["battery_capacity"] = 4000
df_s1["ratings"] = 4.3
df_s1["num_of_ratings"] = 25000
df_s1["sales_price"] = 85000
df_s1["discount_percent"] = 0.05
df_s1["screen_size_Large"] = 1
df_s1["screen_size_Medium"] = 0

In [34]:
df_s1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [38]:
# arr_s1_nor = model_nor.transform(df_s1.drop(columns = "sales"))
arr_s1_nor = model_nor.transform(df_s1)
arr_s1_nor

array([[0.0119438 , 0.49206349, 0.45454545, 1.        , 0.        ,
        0.42307692, 0.625     , 0.05308122, 0.51815842, 0.09302326,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

In [39]:
model_knn.predict(arr_s1_nor[:, 1:]) # 틀린 답안!!!

array([0.00429167])

In [42]:
model_knn_best = KNeighborsRegressor(n_neighbors = best_k)
model_knn_best.fit(X = arr_train_nor[:, 1:],
                   y = arr_train_nor[:, 0 ])
pred_best = model_knn_best.predict(arr_s1_nor[:, 1:])
pred_best

array([0.00132259])

In [48]:
# model_nor.inverse_transform(pred_best)
# model_nor.inverse_transform([pred_best])
arr_s1_nor[0, 0] = pred_best
arr_s1_inv = model_nor.inverse_transform(arr_s1_nor)
arr_s1_inv

array([[6.53333333e-01, 2.56000000e+02, 6.00000000e+00, 4.00000000e+00,
        1.00000000e+00, 4.00000000e+03, 4.30000000e+00, 2.50000000e+04,
        8.50000000e+04, 5.00000000e-02, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [50]:
pd.DataFrame(arr_s1_inv, columns = df_s1.columns)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.653333,256.0,6.0,4.0,1.0,4000.0,4.3,25000.0,85000.0,0.05,1.0,0.0,0.0,0.0,0.0
